In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


sample_submission.csv
test
train



In [2]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 12)
    return files, targets

In [3]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(256, 256))
    # convert PIL.Image.Image type to 3D tensor with shape (256, 256 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 256, 256 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from tqdm import tqdm
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.optimizers import Adam
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [5]:
train_dir = '../input/train/'
test_dir = '../input/test/'

In [6]:
from os import listdir
from os.path import isfile, join

In [7]:
labels = listdir(train_dir)
labels

['Sugar beet',
 'Scentless Mayweed',
 'Small-flowered Cranesbill',
 'Fat Hen',
 'Cleavers',
 'Common Chickweed',
 'Maize',
 'Shepherds Purse',
 'Black-grass',
 'Common wheat',
 'Loose Silky-bent',
 'Charlock']

In [8]:
train_files, train_targets = load_dataset(train_dir)

In [9]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255
train_labels = train_targets

100%|██████████| 4750/4750 [00:53<00:00, 89.00it/s]


In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(train_tensors, train_labels, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_temp.shape)
print(y_temp.shape)


(3800, 256, 256, 3)
(3800, 12)
(950, 256, 256, 3)
(950, 12)


In [12]:
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)


(475, 256, 256, 3)
(475, 12)
(475, 256, 256, 3)


In [13]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(12, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
__________

In [14]:
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "weights.model_rmsprop.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

checkpointer = ModelCheckpoint(filepath='weights.model_rmsprop.hdf5', 
                           verbose=1, save_best_only=True)

In [16]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          epochs=5, batch_size=30, callbacks=callbacks, verbose=1)

Train on 3800 samples, validate on 475 samples
Epoch 1/5
3800/3800 [==============================] - 143s 38ms/step - loss: 1.8630 - acc: 0.3634 - val_loss: 1.5010 - val_acc: 0.4905
Epoch 2/5
3800/3800 [==============================] - 144s 38ms/step - loss: 1.2005 - acc: 0.5887 - val_loss: 1.1782 - val_acc: 0.6400
Epoch 3/5
3800/3800 [==============================] - 139s 37ms/step - loss: 0.9246 - acc: 0.6953 - val_loss: 1.5156 - val_acc: 0.5095
Epoch 4/5
3800/3800 [==============================] - 139s 37ms/step - loss: 0.7181 - acc: 0.7621 - val_loss: 1.2832 - val_acc: 0.6337
Epoch 5/5
3800/3800 [==============================] - 140s 37ms/step - loss: 0.5550 - acc: 0.8213 - val_loss: 1.0835 - val_acc: 0.6842


In [22]:
model.load_weights('weights.model_rmsprop.hdf5')
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) 
               for feature in X_test]

test_list = y_test.argmax(axis=1)

print(f1_score(test_list, predictions, average='weighted')) 
print(accuracy_score(test_list,predictions))

0.65759900889
0.665263157895


In [21]:
print("hello")

hello
